In [3]:
import pandas as pd

In [4]:
! pip install ../

Defaulting to user installation because normal site-packages is not writeable
Processing /home/briarmoss/Documents/Boston_311
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for boston311: filename=boston311-0.2.0-py3-none-any.whl size=23980 sha256=0d6f7aa30f044f6a35e00c978b8e280400b4ade9b8ddb160068a9c2569f3ee1b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vjp_ax5c/wheels/3d/69/ee/0a6ac96b9c09c948fc0e74f2724a9703aa39749a41fa757c9e
Successfully built boston311
  Attempting uninstall: boston311
    Found existing installation: boston311 0.2.0
    Uninstalling boston311-0.2.0:
      Successfully uninstalled boston311-0.2.0


In [5]:
#import boston311 stuff
from boston311 import Boston311LogReg, Boston311EventDecTree, Boston311SurvDecTree, Boston311KerasNN


2023-11-01 08:06:20.179505: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 08:06:20.419972: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-01 08:06:20.421496: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-01 08:06:21.146541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using TensorFlow backend


In [6]:
data = Boston311LogReg(train_date_range={'start':'2010-12-31','end':'2023-12-31'}).load_data()

Checking files_dict
files_dict is None
trying to call get311URLs
trying to get csv URLs
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/e6013a93-1321-4f2a-bf91-8d8a02f1e62f/download/tmpt_2g32xl.csv for year 2023
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/81a7b022-f8fc-4da5-80e4-b160058ca207/download/tmpfm8veglw.csv for year 2022
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/f53ebccd-bc61-49f9-83db-625f209c95f5/download/tmp88p9g82n.csv for year 2021
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/6ff6a6fd-3141-4440-a880-6f60a37fe789/download/tmpcv_10m2s.csv for year 2020
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resource/ea2e4696-4a2d-429c-9807-d02eb92e0222/download/tmpcje3ep_w.csv for year 2019
Found URL: https://data.boston.gov/dataset/8048697b-ad64-4bfc-b090-ee00169f2323/resourc

/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:326: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:326: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:326: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:326: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/home/briarmoss/.local/lib/python3.10/site-packages/boston311/Boston311Model.py:326: DtypeWarning: Columns (13) have mixed types. Specify dtype opt

In [7]:
#save data to csv as all_311_cases.csv
data.to_csv('all_311_cases.csv', index=False)

In [8]:
#save data to a pkl file as all_311_cases.pkl
data.to_pickle('all_311_cases.pkl')

In [9]:
#load empty_response_ids.csv, remove duplicates, and save it again as empty_response_ids.csv
data = pd.read_csv('empty_response_ids.csv')
data = data.drop_duplicates(subset=['case_enquiry_id'])
data.to_csv('empty_response_ids.csv', index=False)

In [10]:
import pandas as pd
import requests
import csv
import time
import json
import os

# Load your CSVs into pandas DataFrames

case_enquiry_id_file = "all_311_cases.csv"
service_request_id_file = "all_311_cases_api.csv"
case_enquiry_id_pkl = "all_311_cases.pkl"
empty_response_file = "empty_response_ids.csv"

#check if pkl file exists
if os.path.exists(case_enquiry_id_pkl):
    print("pkl file exists")
    #check if pkl file modified date is older than csv file
    if os.path.getmtime(case_enquiry_id_pkl) < os.path.getmtime(case_enquiry_id_file):
        print("pkl file is older than csv file")
        #delete pkl file
        os.remove(case_enquiry_id_pkl)
        print("pkl file deleted")
        #read csv file
        df1 = pd.read_csv(case_enquiry_id_file)
        #save csv file as pkl file
        df1.to_pickle(case_enquiry_id_pkl)
    else:
        print("pkl file is newer than csv file")
        df1 = pd.read_pickle(case_enquiry_id_pkl)
else:
    print("pkl file does not exist")
    df1 = pd.read_csv(case_enquiry_id_file)
    df1.to_pickle(case_enquiry_id_pkl)



#file with service_request_id
df2 = pd.read_csv(service_request_id_file)

#check if empty_response_file exists
if os.path.exists(empty_response_file):
    print("empty_response_file exists")
    df3 = pd.read_csv(empty_response_file)


# Identify missing service_request_ids
missing_ids = set(df1['case_enquiry_id']) - set(df2['service_request_id'])

#also subtract the empty_response_file
if df3 is not None:
    missing_ids = missing_ids - set(df3['case_enquiry_id'])



missing_records = df1[df1['case_enquiry_id'].isin(missing_ids)]
missing_records = missing_records.sort_values(by='case_enquiry_id', ascending=False)

#initialize CSV file for empty response IDs
#if the file exists, append to it
if os.path.exists(empty_response_file):
    empty_response_csv_file = open(empty_response_file, 'a', newline='', encoding='utf-8')
    empty_response_csv_writer = csv.writer(empty_response_csv_file)
else:
    empty_response_csv_file = open(empty_response_file, 'w', newline='', encoding='utf-8')
    empty_response_csv_writer = csv.writer(empty_response_csv_file)
    empty_response_csv_writer.writerow(['case_enquiry_id'])

# Initialize CSV file
csv_file_path = service_request_id_file
csv_file = open(csv_file_path, 'a', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)

# Rate limit delay
rate_limit_delay = 6  # 6 seconds to stay within 10 requests per minute
max_exponential_backoff = 3600 

for service_request_id in missing_records['case_enquiry_id']:
    url = f"https://311.boston.gov/open311/v2/requests.json?service_request_id={service_request_id}"

    try:
        response = requests.get(url)
        
        print(f"Fetching data for service_request_id {service_request_id}")

        if response.status_code == 200:
            try:
                data = json.loads(response.text)
                if not data:  # if data is empty
                    empty_response_csv_writer.writerow([service_request_id])
                    empty_response_csv_file.flush()
                    print(f"Empty response for service_request_id {service_request_id}")
                else:
                    for record in data:
                        #print all the data
                        #print(record.get('service_request_id'), record.get('status'), record.get('service_name'), record.get('service_code'), record.get('description'), record.get('requested_datetime'), record.get('updated_datetime'), record.get('address'), record.get('lat'), record.get('long'), record.get('token'))
                        csv_writer.writerow([
                            record.get('service_request_id'),
                            record.get('status'),
                            record.get('service_name'),
                            record.get('service_code'),
                            record.get('description'),
                            record.get('requested_datetime'),
                            record.get('updated_datetime'),
                            record.get('address'),
                            record.get('lat'),
                            record.get('long'),
                            record.get('token')
                        ])
                        csv_file.flush()
                        print(f"Data written for service_request_id {service_request_id}")
                    rate_limit_delay = 6
            except json.JSONDecodeError:
                print(f"Failed to decode JSON for service_request_id {service_request_id}")
                # Double the delay time after failure
                rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)
                print(f"Waiting {rate_limit_delay} seconds before retrying")
        else:
            print(f"Failed to fetch data for service_request_id {service_request_id}")
            # Double the delay time after failure
            rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)
            print(f"Waiting {rate_limit_delay} seconds before retrying")   
    except requests.exceptions.ConnectionError:
        print(f"Failed to connect to {url}")
        # Double the delay time after failure
        rate_limit_delay = min(rate_limit_delay * 2, max_exponential_backoff)
        print(f"Waiting {rate_limit_delay} seconds before retrying")

    # Rate limiting
    time.sleep(rate_limit_delay)

# Close CSV file
csv_file.close()


pkl file exists
pkl file is newer than csv file
empty_response_file exists
Fetching data for service_request_id 101005149461
Data written for service_request_id 101005149461
Fetching data for service_request_id 101005149460
Empty response for service_request_id 101005149460
Fetching data for service_request_id 101005149458
Data written for service_request_id 101005149458
Fetching data for service_request_id 101005149457
Data written for service_request_id 101005149457
Fetching data for service_request_id 101005149455
Empty response for service_request_id 101005149455
Fetching data for service_request_id 101005149452
Data written for service_request_id 101005149452
Fetching data for service_request_id 101005149449
Empty response for service_request_id 101005149449
Fetching data for service_request_id 101005149448
Data written for service_request_id 101005149448
Fetching data for service_request_id 101005149446
Empty response for service_request_id 101005149446
Fetching data for service_

KeyboardInterrupt: 